# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [4]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


By defining the ID column as the index column it is trivial to return an artist by just querying the ID.

In [5]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [6]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [9]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))

Princess Bride, The (1987)


In [10]:
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))

Error obtaining title


# _The ^^ above ^^ cell should show an error_

In [11]:
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Error obtaining title


# _The ^^ above ^^ cell should show an error_

Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

In [12]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [13]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Braveheart (1995)
Terminator, The (1984)
Fugitive, The (1993)
Speed (1994)
Crimson Tide (1995)
GoldenEye (1995)
Batman (1989)
Clear and Present Danger (1994)
True Lies (1994)
Mask, The (1994)
Die Hard: With a Vengeance (1995)
In the Line of Fire (1993)
Lion King, The (1994)
Ghost (1990)
Forrest Gump (1994)
Apollo 13 (1995)
Cliffhanger (1993)
Star Trek: Generations (1994)
Firm, The (1993)
Die Hard (1988)
Seven (a.k.a. Se7en) (1995)
Indiana Jones and the Last Crusade (1989)
Mission: Impossible (1996)
Mrs. Doubtfire (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for our sample collection of artists render in a nice dataframe. Again, let's create a helper function to achieve this.

In [14]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our SIMS campaign. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [15]:
samples = items_df.sample(5)
samples

,title
movieId,
1257,Better Off Dead... (1985)
2275,Six-String Samurai (1998)
6212,Bringing Down the House (2003)
2164,Surf Nazis Must Die (1987)
7841,Children of Dune (2003)


In [16]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,Better Off Dead... (1985),Six-String Samurai (1998),Bringing Down the House (2003),Surf Nazis Must Die (1987),Children of Dune (2003)
0,Man on Wire (2008),Kick-Ass (2010),"Shawshank Redemption, The (1994)","Shawshank Redemption, The (1994)","Shawshank Redemption, The (1994)"
1,Major League (1989),Iron Man 2 (2010),Forrest Gump (1994),Forrest Gump (1994),Forrest Gump (1994)
2,Legend (1985),NaN,Pulp Fiction (1994),Pulp Fiction (1994),Pulp Fiction (1994)
3,Anne of Green Gables: The Sequel (a.k.a. Anne ...,NaN,"Silence of the Lambs, The (1991)","Silence of the Lambs, The (1991)","Silence of the Lambs, The (1991)"
4,Jezebel (1938),NaN,"Matrix, The (1999)","Matrix, The (1999)","Matrix, The (1999)"
5,Anne of Green Gables (1985),NaN,Braveheart (1995),Braveheart (1995),Braveheart (1995)
6,She's Out of Control (1989),NaN,Schindler's List (1993),Schindler's List (1993),Schindler's List (1993)
7,Meet the Spartans (2008),NaN,Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977)
8,Beautiful (2000),NaN,Jurassic Park (1993),Jurassic Park (1993),Jurassic Park (1993)
9,Bounce (2000),NaN,Terminator 2: Judgment Day (1991),Terminator 2: Judgment Day (1991),Terminator 2: Judgment Day (1991)


You may notice that a lot of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions). This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [17]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[346, 75, 247]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [18]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [19]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,346,75,247
0,Blade Runner (1982),Star Wars: Episode V - The Empire Strikes Back...,"Sound of Music, The (1965)"
1,"Terminator, The (1984)",Raiders of the Lost Ark (Indiana Jones and the...,Singin' in the Rain (1952)
2,Rear Window (1954),Monty Python and the Holy Grail (1975),Lady and the Tramp (1955)
3,Ghostbusters (a.k.a. Ghost Busters) (1984),Rocky (1976),Cinderella (1950)
4,"Road Warrior, The (Mad Max 2) (1981)","Wizard of Oz, The (1939)","Little Mermaid, The (1989)"
5,Taxi Driver (1976),"Godfather: Part II, The (1974)",My Fair Lady (1964)
6,E.T. the Extra-Terrestrial (1982),Indiana Jones and the Last Crusade (1989),It's a Wonderful Life (1946)
7,"Professional, The (Le professionnel) (1981)",Back to the Future (1985),"King and I, The (1956)"
8,Star Wars: Episode V - The Empire Strikes Back...,"Princess Bride, The (1987)",Meet Me in St. Louis (1944)
9,Alien (1979),"Shining, The (1980)","Jungle Book, The (1967)"


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

Now lets apply item filters to see recommendations for one of these users within a genre


In [20]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [21]:
user

247

You can see the recommendations for movies within a given genre. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [22]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

,Fantasy,Western,Comedy,Animation,Action,Thriller,Documentary,1970s
0,Cinderella (1950),Oklahoma! (1955),Singin' in the Rain (1952),Lady and the Tramp (1955),North by Northwest (1959),North by Northwest (1959),Monterey Pop (1968),Robin Hood (1973)
1,It's a Wonderful Life (1946),Seven Brides for Seven Brothers (1954),Lady and the Tramp (1955),Cinderella (1950),Speed (1994),Vertigo (1958),Sympathy for the Devil (1968),Grease (1978)
2,Chitty Chitty Bang Bang (1968),"Fistful of Dollars, A (Per un pugno di dollari...","Little Mermaid, The (1989)","Little Mermaid, The (1989)","Blues Brothers, The (1980)",To Catch a Thief (1955),Stop Making Sense (1984),"Muppet Movie, The (1979)"
3,"Sword in the Stone, The (1963)","American Tail: Fievel Goes West, An (1991)",My Fair Lady (1964),"Jungle Book, The (1967)","Incredibles, The (2004)",Speed (1994),In the Realms of the Unreal (2004),"Rocky Horror Picture Show, The (1975)"
4,Peter Pan (1953),Annie Get Your Gun (1950),"Jungle Book, The (1967)",Shrek 2 (2004),Ben-Hur (1959),Rear Window (1954),Hoop Dreams (1994),"Aristocats, The (1970)"
5,Beauty and the Beast (1991),"Good, the Bad and the Ugly, The (Buono, il bru...",Shrek 2 (2004),Dumbo (1941),Scott Pilgrim vs. the World (2010),Notorious (1946),Why Man Creates (1968),A Cosmic Christmas (1977)
6,Alice in Wonderland (1951),Stagecoach (1939),Chitty Chitty Bang Bang (1968),Mulan (1998),Raiders of the Lost Ark (Indiana Jones and the...,Charade (1963),Pearl Jam Twenty (2011),Annie Hall (1977)
7,Brigadoon (1954),"Big Country, The (1958)","Music Man, The (1962)","Sword in the Stone, The (1963)","Great Escape, The (1963)",Rebecca (1940),Seven Up! (1964),Pete's Dragon (1977)
8,Fantasia (1940),Rio Grande (1950),"Night at the Opera, A (1935)",Peter Pan (1953),"Fistful of Dollars, A (Per un pugno di dollari...",Dracula (Bram Stoker's Dracula) (1992),"Decline of Western Civilization, The (1981)",Charlotte's Web (1973)
9,"Wizard of Oz, The (1939)",Love Me Tender (1956),Mulan (1998),Beauty and the Beast (1991),Zootopia (2016),Seven (a.k.a. Se7en) (1995),Madonna: Truth or Dare (1991),Bedknobs and Broomsticks (1971)


The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [23]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:835319576252:event-tracker/1e2b508b
b5b715b6-6d9d-4166-a64b-e320f968c84d


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [24]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [25]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,247
0,"Sound of Music, The (1965)"
1,Singin' in the Rain (1952)
2,Lady and the Tramp (1955)
3,Cinderella (1950)
4,"Little Mermaid, The (1989)"
5,My Fair Lady (1964)
6,It's a Wonderful Life (1946)
7,"King and I, The (1956)"
8,Meet Me in St. Louis (1944)
9,"Jungle Book, The (1967)"


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [26]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [27]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    time.sleep(5)
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
user_recommendations_df

sending event click for Super Size Me (2004)
sending event click for Transporter 3 (2008)
sending event click for Stoning of Soraya M., The (2008)


,247,Super Size Me (2004),Transporter 3 (2008),"Stoning of Soraya M., The (2008)"
0,"Sound of Music, The (1965)","Sound of Music, The (1965)",Shrek 2 (2004),Shrek 2 (2004)
1,Singin' in the Rain (1952),Singin' in the Rain (1952),"40-Year-Old Virgin, The (2005)",Casino Royale (2006)
2,Lady and the Tramp (1955),Lady and the Tramp (1955),Lost in Translation (2003),Little Miss Sunshine (2006)
3,Cinderella (1950),Cinderella (1950),Eternal Sunshine of the Spotless Mind (2004),Avatar (2009)
4,"Little Mermaid, The (1989)","Little Mermaid, The (1989)",Howl's Moving Castle (Hauru no ugoku shiro) (2...,Mr. & Mrs. Smith (2005)
5,My Fair Lady (1964),My Fair Lady (1964),Bruce Almighty (2003),Pirates of the Caribbean: Dead Man's Chest (2006)
6,It's a Wonderful Life (1946),It's a Wonderful Life (1946),Finding Nemo (2003),Mulan (1998)
7,"King and I, The (1956)","King and I, The (1956)","Terminal, The (2004)",Stardust (2007)
8,Meet Me in St. Louis (1944),Meet Me in St. Louis (1944),Love Actually (2003),Harry Potter and the Order of the Phoenix (2007)
9,"Jungle Book, The (1967)","Jungle Book, The (1967)","Good bye, Lenin! (2003)",Star Trek (2009)


In the cell above, the first column after the index is the user's default recommendations from HRNN, and each column after that has a header of the artist that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies, and you should see a more pronounced impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) For VOD shelves you could move a title from "Top picks for you" to a "Watch again", the watch again recommendations will be based on the users current interactions, but only recommend titles that have already been watched.


In [28]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Harry Potter and the Order of the Phoenix (2007),Shrek 2 (2004)
1,Star Trek (2009),Casino Royale (2006)
2,"Bourne Ultimatum, The (2007)",Little Miss Sunshine (2006)
3,"Incredibles, The (2004)",Avatar (2009)
4,NaN,Mr. & Mrs. Smith (2005)
5,NaN,Pirates of the Caribbean: Dead Man's Chest (2006)
6,NaN,Mulan (1998)
7,NaN,Stardust (2007)
8,NaN,Iron Man (2008)
9,NaN,"Jungle Book, The (1994)"


now lets send a watch event in for 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [29]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)


sending event watch for Shrek 2 (2004)
sending event watch for Casino Royale (2006)
sending event watch for Little Miss Sunshine (2006)
sending event watch for Avatar (2009)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [30]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Avatar (2009),"Sound of Music, The (1965)"
1,Star Trek (2009),Lady and the Tramp (1955)
2,V for Vendetta (2006),Cinderella (1950)
3,300 (2007),"Little Mermaid, The (1989)"
4,Harry Potter and the Order of the Phoenix (2007),My Fair Lady (1964)
5,Shrek 2 (2004),It's a Wonderful Life (1946)
6,Batman Begins (2005),"King and I, The (1956)"
7,Casino Royale (2006),Meet Me in St. Louis (1944)
8,"Lion King, The (1994)",Doctor Dolittle (1967)
9,"Prestige, The (2006)",Shrek 2 (2004)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, howeverr you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [31]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [32]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,Idle Hands (1999)
1,Heartless (2009)
2,Wadjda (2012)
3,Play Misty for Me (1971)
4,"Jazz Singer, The (1927)"
5,Great Expectations (1998)
6,"Last Samurai, The (2003)"
7,Trading Places (1983)
8,Porky's II: The Next Day (1983)
9,Shaun the Sheep Movie (2015)


Then make the personalized ranking API call.

In [33]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': 'f1f41c69-7541-45e8-a31e-f92e4c799bd5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Tue, 15 Sep 2020 03:33:12 GMT',
   'x-amzn-requestid': 'f1f41c69-7541-45e8-a31e-f92e4c799bd5',
   'content-length': '1436',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '7143', 'score': 0.9174433},
  {'itemId': '2606', 'score': 0.0124089},
  {'itemId': '65585', 'score': 0.0094577},
  {'itemId': '1894', 'score': 0.0076884},
  {'itemId': '1513', 'score': 0.0066342},
  {'itemId': '66544', 'score': 0.0065695},
  {'itemId': '101070', 'score': 0.0062184},
  {'itemId': '6860', 'score': 0.0053613},
  {'itemId': '2605', 'score': 0.005054},
  {'itemId': '2498', 'score': 0.0050442},
  {'itemId': '3039', 'score': 0.0037912},
  {'itemId': '131656', 'score': 0.0035436},
  {'itemId': '2253', 'score': 0.0027009},
  {'itemId': '1735', 'score': 0.0023023},
  {'itemId': '2735', 'score': 0.0019111

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [34]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,Idle Hands (1999),"Last Samurai, The (2003)"
1,Heartless (2009),Idle Hands (1999)
2,Wadjda (2012),Bride Wars (2009)
3,Play Misty for Me (1971),Six Days Seven Nights (1998)
4,"Jazz Singer, The (1927)",Romy and Michele's High School Reunion (1997)
5,Great Expectations (1998),Nuremberg (2000)
6,"Last Samurai, The (2003)",Wadjda (2012)
7,Trading Places (1983),Mobsters (1991)
8,Porky's II: The Next Day (1983),Entrapment (1999)
9,Shaun the Sheep Movie (2015),My Favorite Martian (1999)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [35]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [36]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "346"}
{"userId": "75"}
{"userId": "247"}


Upload the file to S3 and save the path as a variable for later.

In [37]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://835319576252personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [38]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://835319576252personalizepocvod/


Now just make the call to kick off the batch export process.

In [39]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [43]:
%%time

current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  04:01:17 AM
DatasetInferenceJob: ACTIVE
Import Completed on:  04:01:17 AM
CPU times: user 4.37 ms, sys: 114 µs, total: 4.48 ms
Wall time: 29.9 ms


In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `04_Clean_Up_Resources.ipynb`.

In [ ]:
%store event_tracker_arn
%store batchInferenceJobArn